# Welcome to the HNN-Core ERP tutorial!
This tutorial follows the [HNN-GUI tutorial](https://jonescompneurolab.github.io/hnn-tutorials/erp/erp) with python commands instead of through a graphical user interface. We'll begin with some background on the experimental data used in this tutorial, and then give you a quick introduction to the HNN-Core API. Finally, we'll show you how to adjusting model parameters to test specific hypotheses using HNN simulations.

In order to understand the workflow and initial parameter sets provided with this tutorial, we must first briefly describe prior studies that led to the creation of the provided data and evoked response parameter set that you will work with. This tutorial is based on results from our 2007 study where we recorded and simulated tactile evoked responses source localized to the primary somatosensory cortex (SI) [1].

In our 2007 study, we investigated the early evoked activity (0-175 ms) elicited by a brief tap to the D3 digit and source localized to an an equivalent current dipole in the contralateral hand area of the primary somatosensory cortex (SI) [1]. The strength of the tap was set at either suprathreshold (100% detection probability) or perceptual  threshold (50% detection) levels (see Figure 1, left panel below). Note, to be precise, this data represents source localized event related field (ERF) activity because it was collected using MEG. We use the terminology ERP for simplicity, since the primary current dipoles generating evoked fields and potentials are the same.

We found that we could reproduce evoked responses that accurately reflected the recorded waveform in our neocortical model from a layer specific sequence of exogenous excitatory synaptic drive to the local SI circuit, see Figure 1right panel below. This drive consisted of “feedforward” / proximal input at ~25 ms post-stimulus, followed by “feedback” / distal input at ~60 ms, followed by a subsequent “feedforward” / proximal input at ~125 ms (Gaussian distribution of input times on each simulated trial). This sequence of drive generated spiking activity and intracellular dendritic current flow in the pyramidal neuron dendrites to reproduce the current dipole signal. This sequence of drive can be interpreted as initial “feedforward” input from the lemniscal thalamus, followed by “feedback” input from higher order cortex or non-lemniscal thalamus, followed by a re-emergent leminsical thalamic drive. Intracranial recordings in non-human primates motivated and supported this assumption [2].

In our model, the exogenous driving inputs were simulated as predefined trains of action potentials (pre-synaptic spikes) that activated excitatory synapses in the local cortical circuit in proximal and distal projection patterns (i.e. feedforward, and feedback, respectively, as shown schematically in Figure 1 right). The number, timing and strength (post-synaptic conductance) of the driving spikes were manually adjusted in the model until a close representation of the data was found (all other model parameters were tuned and fixed based on the morphology, physiology and connectivity within layered neocortical circuits [1]. Note, a scaling factor was applied to net dipole output to match to the magnitude of the recorded ERP data and used to predict the number of neurons contributing to the recorded ERP (purple circle, Figure 1, right panel). The dipole units were in nAm, with a one-to-one comparison between data and model output due to the biophysical detail in our model.

<div class="stylefig">

### Figure 1

<a href="https://raw.githubusercontent.com/jonescompneurolab/hnn-tutorials/master/erp/images/image8.png"><img class="imgcenter100" src="https://raw.githubusercontent.com/jonescompneurolab/hnn-tutorials/master/erp/images/image8.png" alt="image8" style="max-width:650px;"/></a>

<p style="text-align:justify;"> Adapted from Jones et al. 2007 [1]. Comparison of SI evoked response in experiment and neural model simulation. Left: MEG data showing tactile evoked response (ERP) source localized to the hand area of SI. Red: suprathreshold stimulation; Blue: Threshold stimulation (avg. n=100 trials). Right: Neural model simulation depicting proximal/distal inputs needed to replicate the ERP waveform (avg. n=25 trials) </p>
</div>

In summary, to simulate the SI evoked response, a sequence of exogenous excitatory synaptic drive was simulated (by creating presynaptic spikes that activate layer specific synapses in the neocortical network) consisting of proximal drive at ~25 ms, followed by distal drive at ~60 ms, followed by a second proximal drive at ~122 ms. Given this background information, we can now walk you through the steps of simulating a similar ERP, using a subset of the data shown in Figure 1.

### References
1. Jones, S. R., Pritchett, D. L., Stufflebeam, S. M., Hämäläinen, M. & Moore, C. I. Neural correlates of tactile detection: a combined magnetoencephalography and biophysically based computational modeling study. J. Neurosci. 27, 10751–10764 (2007).

2. Cauller, L. J. & Kulics, A. T. The neural basis of the behaviorally relevant N1 component of the somatosensory-evoked potential in SI cortex of awake monkeys: evidence that backward cortical projections signal conscious touch sensation. Exp. Brain Res. 84, 607–619 (1991).

## 1. Importing python libraries and loading data

In [ ]:
import os.path as op

import numpy as np
import matplotlib.pyplot as plt

import hnn_core
from hnn_core import jones_2009_model, simulate_dipole, MPIBackend, JoblibBackend, average_dipoles
from hnn_core.viz import plot_dipole, plot_tfr_morlet

Let's retrieve and load the experimental dipole. This is the average SI threshold-level evoked response from detected (yes) trials.

In [ ]:
from urllib.request import urlretrieve
data_url = ('https://raw.githubusercontent.com/jonescompneurolab/hnn/master/'
            'data/MEG_detection_data/yes_trial_S1_ERP_all_avg.txt')
urlretrieve(data_url, 'yes_trial_S1_ERP_all_avg.txt')

Then we read the dipole using ``hnn-core``

In [ ]:
from hnn_core import read_dipole

exp_dpl = read_dipole('yes_trial_S1_ERP_all_avg.txt')
exp_dpl.plot();

## 2. Running a simulation

Instantiate the network model

In [ ]:
net = jones_2009_model()

Now we add a proximal drive at around 26 ms. The proximal drive represents a feedforward input from the lemniscal thalamus. <a href="https://raw.githubusercontent.com/jonescompneurolab/hnn-under_the_hood/master/html-styling/images/prox-drive.png"><img class="imgcenter100" src="https://raw.githubusercontent.com/jonescompneurolab/hnn-under_the_hood/master/html-styling/images/prox-drive.png" alt="image8" style="max-width:650px;"/></a>

In [ ]:
weights_ampa_p1 = {'L2_basket': 0.08831, 'L2_pyramidal': 0.01525,
                   'L5_basket': 0.19934, 'L5_pyramidal': 0.00865}
synaptic_delays_prox = {'L2_basket': 0.1, 'L2_pyramidal': 0.1,
                        'L5_basket': 1., 'L5_pyramidal': 1.}

# all NMDA weights are zero; pass None explicitly
net.add_evoked_drive(
    'evprox1', mu=26.61, sigma=2.47, numspikes=1, weights_ampa=weights_ampa_p1,
    weights_nmda=None, location='proximal',
    synaptic_delays=synaptic_delays_prox, event_seed=4)



We can check that the drives have been added by looking at

In [ ]:
net.external_drives

Then we add a distal drive at around 63 ms. This represents "feedback" input from higher-order cortex. <a href="https://raw.githubusercontent.com/jonescompneurolab/hnn-under_the_hood/master/html-styling/images/dist-drive.png"><img class="imgcenter100" src="https://raw.githubusercontent.com/jonescompneurolab/hnn-under_the_hood/master/html-styling/images/dist-drive.png" alt="image8" style="max-width:650px;"/></a>

In [ ]:
weights_ampa_d1 = {'L2_basket': 0.006562, 'L2_pyramidal': .000007,
                   'L5_pyramidal': 0.142300}
weights_nmda_d1 = {'L2_basket': 0.019482, 'L2_pyramidal': 0.004317,
                   'L5_pyramidal': 0.080074}
synaptic_delays_d1 = {'L2_basket': 0.1, 'L2_pyramidal': 0.1,
                      'L5_pyramidal': 0.1}

net.add_evoked_drive(
    'evdist1', mu=63.53, sigma=3.85, numspikes=1, weights_ampa=weights_ampa_d1,
    weights_nmda=weights_nmda_d1, location='distal',
    synaptic_delays=synaptic_delays_d1, event_seed=4)

Finally, we add a second proximal drive at around 137 ms that represents a re-emergent lemniscal thalamic drive. 
Intracranial recordings in non-human primates motivated and supported this assumption [2].

In [ ]:
weights_ampa_p2 = {'L2_basket': 0.000003, 'L2_pyramidal': 1.438840,
                   'L5_basket': 0.008958, 'L5_pyramidal': 0.684013}

# all NMDA weights are zero; omit weights_nmda (defaults to None)
net.add_evoked_drive(
    'evprox2', mu=137.12, sigma=8.33, numspikes=1,
    weights_ampa=weights_ampa_p2, location='proximal',
    synaptic_delays=synaptic_delays_prox, event_seed=4)

Now, we simulate the dipole with just one trial for now

In [ ]:
dpls = simulate_dipole(net, tstop=170., n_trials=1)

The simulation is a bit slow. If we want to speed it up, we can use MPI. It's a protocol that splits the simulation across neurons. You might need to follow a few extra installation steps to install MPI dependencies if you wish to run `MPIBackend` on your machine (see [here](https://jonescompneurolab.github.io/hnn-core/stable/parallel.html)).

In [ ]:
# simulate dipole with a specific parallel backend (2 trials)
# we'll use MPIBackend for the remainder of this tutorial as it is the fastest
with MPIBackend(n_procs=4):
    dpls = simulate_dipole(net, tstop=170., n_trials=2)
    
# If you don't have the OpenMPI and mpi4py installed on you machine,
# you can alternatively use JoblibBackend (uncomment lines below)
# with JoblibBackend(n_jobs=2):
#    dpls = simulate_dipole(net, tstop=170., n_trials=2)

## 3. Plotting and visualization

The simulation returns a list of dipoles with length equal to number of trials.

In [ ]:
dpls

First we'll plot the raw waveform that is unsmoothed and unscaled

In [ ]:
plot_dipole(dpls);

Now, note that we simulated a network of 10 x 10 pyramidal cells but human cortex contains much more neurons and the MEG signals usually represent synchronous activation of at least ~50,000 pyramidal neurons. Hence, we must scale the signal to match the source localized data. Averaging across so many neurons will also smooth the signal. Hence for a better comparison, we should smooth and scale

In [ ]:
window_len, scaling_factor = 30, 3000
for dpl in dpls:
    dpl.smooth(window_len).scale(scaling_factor)

plot_dipole(dpls);

Let's see what the simulated dipole looks like compared to the experimental dipole. We will also visualize the spike histograms of the drive for easy interpretation.

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(6, 6), constrained_layout=True)

avg_dpl = average_dipoles(dpls)
exp_dpl.plot(ax=axes[0], show=False)
avg_dpl.plot(ax=axes[0], show=False)

axes[0].legend(['experimental', 'simulated avgerage'])

# driving input spike histogram
net.cell_response.plot_spikes_hist(ax=axes[1]);

We can also plot the dipole generated by the pyramidal cells in L2/3 and L5, respectively. Looking at this plot, we can conclude that the aggregate activity mainly originates from the L5 cells.

In [ ]:
# plot layer-specific dipoles
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(6, 6), constrained_layout=True, sharey=True)

for idx, layer in enumerate(['agg', 'L2', 'L5']):
    plot_dipole(avg_dpl, ax=axes[idx], layer=layer, show=False)

# driving input spike histogram
net.cell_response.plot_spikes_hist(spike_types=['evprox', 'evdist']);

We can also plot the spiking activity stored in `net.cell_response`:

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(6, 6), constrained_layout=True)
net.cell_response.plot_spikes_raster(ax=axes[0], show=False);
net.cell_response.plot_spikes_hist(ax=axes[1],
                                   spike_types=['L5_pyramidal', 'L2_pyramidal', 'L5_basket', 'L2_basket'],
                                   show=False);

If we are looking at rhythmic activity, we may also be interested in the time-frequency spectrogram of the waveform

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(6, 6),
                         constrained_layout=True)

exp_dpl.plot(ax=axes[0], show=False)
avg_dpl.plot(ax=axes[0], show=False)
axes[0].legend(['experimental', 'simulated avg.'])

# plot the TFR spectrogram
# note that it won't be very good given that our simulation is not long enough to resolve low frequencies
freqs = np.arange(10, 60)
n_cycles = 10 / freqs
plot_tfr_morlet(dpls, ax=axes[1], freqs=freqs, n_cycles=n_cycles);

## 4. Network properties

Recall that we originally created the network by calling `net = jones_2009_model()`. This object represents the laminar SI model as originally presented in Jones et al. (2009), and gets modified by running the simulation. Let's visualize where the cell somas are positioned within the network in 3D space.

In [ ]:
net

In [ ]:
net.plot_cells();

Cell connectivity is defined for each cell type (`L2_basket`, `L2_pyramidal`, `L5_basket`, and `L5_pyramidal`) and receptor (`ampa`, `nmda`, `gabaa`, and `gabab`)

In [ ]:
net.connectivity[:2]

The cell properties are also visible

In [ ]:
net.cell_types['L5_pyramidal'].synapses

Our L5 pyramidal neurons have 9 compartments

In [ ]:
net.cell_types['L5_pyramidal'].sections.keys()

In [ ]:
net.cell_types['L5_pyramidal'].plot_morphology();

We can see the parameters of a particular compartment

In [ ]:
net.cell_types['L5_pyramidal'].sections['apical_trunk']

and the synapses on to this section

In [ ]:
net.cell_types['L5_pyramidal'].sections['apical_trunk'].syns

## 5. Creating higher-level functions that fit YOUR workflow

Here we'll define two new functions that will help streamline further exploration in this tutorial.

In [ ]:
# a new function that simulates and smooths dipoles
def simulate_and_smooth(net_with_drives, n_trials=1, smoothing_window=30, scaling_factor=3000):
    with MPIBackend(n_procs=4):
        dpls = simulate_dipole(net_with_drives, tstop=170., n_trials=n_trials)

    for dpl in dpls:
        dpl.smooth(window_len).scale(scaling_factor)
    return dpls

In [ ]:
# a new function that generates standard simulation plots
def plot_sim(net, dpls, exp_dpl=exp_dpl):
    fig, axes = plt.subplots(2, 1, sharex=True, figsize=(6, 6), constrained_layout=True)

    exp_dpl.plot(ax=axes[0], show=False)
    average_dipoles(dpls).plot(ax=axes[0], show=False)
    axes[0].legend(['experimental', 'sim. avg.'])
    fig = net.cell_response.plot_spikes_hist(ax=axes[1],
                                             spike_types=['evprox', 'evdist'])

## 6. Adjusting parameters

### 6.1 Changing the timing and strength (post-synaptic conductance) of the evoked inputs
For this part of the tutorial, we’ll load a different experimental dataset, at first keeping the simulation parameters that we started out with. The new experimental data represents the evoked response (ER) from non-detected threshold level stimuli in the experiment described in the introduction section above [1].

Once we load the new ER waveform, notice that the timing and magnitude of the peaks in this new  are different than for the ERs that were detected.

In [ ]:
# retrieve and load the experimental dipole
data_url = ('https://raw.githubusercontent.com/jonescompneurolab/hnn/master/'
            'data/MEG_detection_data/no_trial_S1_ERP_all_avg.txt')
urlretrieve(data_url, 'no_trial_S1_ERP_all_avg.txt')
exp_dpl_nd = read_dipole('no_trial_S1_ERP_all_avg.txt')

# compare to previous experimental dipole
fig = exp_dpl.plot(show=False)
ax = fig.gca()
fig = exp_dpl_nd.plot(ax=ax, show=False)
ax.legend(['detected', 'non-detected'])
plt.show()

Given that the non-detection ER (orange) has peaks that occure slightly later and are decreased in magnitude, we will test the following hypotheses about its origin in relation to the detected ER:
1. The non-detected ER represents a decrease in the strength of the inputs that create the evoked response
2. The non-detected ER can be produced by a more delayed arrival time of these inputs to the network

Recalling our previous drive values for the "detected" case....

In [ ]:
# early proximal
weights_ampa_p1 = {'L2_basket': 0.08831, 'L2_pyramidal': 0.01525,
                   'L5_basket': 0.19934, 'L5_pyramidal': 0.00865}
mu_p1, sigma_p1 = 26.61, 2.47

# distal
weights_ampa_d1 = {'L2_basket': 0.006562, 'L2_pyramidal': .000007,
                   'L5_pyramidal': 0.142300}
weights_nmda_d1 = {'L2_basket': 0.019482, 'L2_pyramidal': 0.004317,
                   'L5_pyramidal': 0.080074}
mu_d1, sigma_d1 = 63.53, 3.85

# late proximal 
weights_ampa_p2 = {'L2_basket': 0.000003, 'L2_pyramidal': 1.438840,
                   'L5_basket': 0.008958, 'L5_pyramidal': 0.684013}
mu_p2, sigma_p2 = 137.12, 8.33

Starting with a clean network (i.e., one without any drives added to it), we can modify the parameters associated with each drive according to our hypotheses

In [ ]:
# re-instantiate the network
net = jones_2009_model()

In [ ]:
# distal
weights_ampa_d1['L2_basket'] += 6.28098
weights_ampa_d1['L2_pyramidal'] += 2.07921
weights_ampa_d1['L5_pyramidal'] += -0.13717

mu_d1 += 8.57
sigma_d1 += 1.15

net.add_evoked_drive(
    'evdist1', mu=mu_d1, sigma=sigma_d1, numspikes=1, weights_ampa=weights_ampa_d1,
    location='distal', synaptic_delays=synaptic_delays_d1, event_seed=4)

In [ ]:
# early proximal
weights_ampa_p1['L2_basket'] += -0.03280
weights_ampa_p1['L2_pyramidal'] += -0.00450
weights_ampa_p1['L5_basket'] += 0.00347
weights_ampa_p1['L5_pyramidal'] += -0.00841

mu_p1 += 13.99
sigma_p1 += 0.03

net.add_evoked_drive(
    'evprox1', mu=mu_p1, sigma=sigma_p1, numspikes=1, weights_ampa=weights_ampa_p1,
    location='proximal', synaptic_delays=synaptic_delays_prox, event_seed=4)

In [ ]:
# late proximal
weights_ampa_p2['L2_basket'] += 0.00399
weights_ampa_p2['L2_pyramidal'] += -1.38763
weights_ampa_p2['L5_basket'] += 0.00409
weights_ampa_p2['L5_pyramidal'] += -0.30884

mu_p2 += 7.58
sigma_p2 += 5.87

net.add_evoked_drive(
    'evprox2', mu=mu_p2, sigma=sigma_p2, numspikes=1,
    weights_ampa=weights_ampa_p2, location='proximal',
    synaptic_delays=synaptic_delays_prox, event_seed=4)

Run the simulation and see if our changes to the drive parameters result in a good match to the new (non-detected) experimental ER

In [ ]:
dpls = simulate_and_smooth(net, n_trials=1)

fig = plot_sim(net, dpls, exp_dpl=exp_dpl_nd)

### 6.2 Drive parameter optimization
Instead of manually tuning the drive parameters to match the non-detected ERP, now we're going try optimizing the original parameters using an algorithmic approach.

First, we'll load the `default.json` parameter file distributed with HNN-Core. This is the full parameter set that we used to simulate the detected ER at the beginning of this tutorial.

In [ ]:
from hnn_core import read_params

# find root directory and set the desired file name
hnn_core_root = op.join(op.dirname(hnn_core.__file__))
params_fname = op.join(hnn_core_root, 'param', 'default.json')

# load in the parameter set
params = read_params(params_fname)

When we instantiate the network model this time, we add the drives according to the parameter file

In [ ]:
# load network and add drives according to param file
net = jones_2009_model(params=params, add_drives_from_params=True)

# simulate
dpls = simulate_and_smooth(net, n_trials=1)

# plot simulation against the non-detect ER experimental data
plot_sim(net, dpls, exp_dpl=exp_dpl_nd);

Running the optimization routine with only 3 iterations (`maxiter=3`) will not improve the fit very much, but a slight improvement may be noticeable

In [ ]:
from hnn_core.optimization import optimize_evoked

# select the first (and only) dipole contained in the dpls list
initial_dpl = dpls[0]

# run optimization routine!!
with MPIBackend(n_procs=4):
    params_optim = optimize_evoked(params, exp_dpl, initial_dpl,
                                   maxiter=3,
                                   scale_factor=3000,
                                   smooth_window_len=30)

Finally, let's run a simulation with the optimized parameter set and see if the simulated dipole better matches the empirical dipole

In [ ]:
net_optim = jones_2009_model(params=params_optim, add_drives_from_params=True)

dpls = simulate_and_smooth(net_optim, n_trials=1)
plot_sim(net_optim, dpls, exp_dpl=exp_dpl_nd);

## 7 Exercises for further exploration (optional)

### 7.1 Asynchronous vs. synchronous drive
By default, HNN-Core provides an independently sampled spike train for each cell targetted by an exogenous drive. We call this an asynchonous drive because different cells receive asynchronous driving spikes. Let's try to make the exogenous driving inputs to the cells synchronous and see what happens. This can be accomplished in the model my specifying that each drive consists of only 1 artificial cell (i.e., a single source of spikes for the network) and delivers spike trains that are not cell-specific (i.e., they target all network cells).

In [ ]:
# using the previously defined synaptic weight and delay values,
# we'll create a new network and add the same pattern of drives as before except with
# n_drive_cells=1 and cell_specific=False

net_sync = jones_2009_model()

#n_drive_cells=1
#cell_specific=False

# [add the drives here]

In [ ]:
# now simulate with our custom higher-level functions
#dpls = simulate_and_smooth(net_sync)

#fig = plot_sim(net_sync, dpls)

### 7.2 Adjusting other parameters

* Try adjusting the duration and strength of the distal drive; how does this affect the simulation?

* Try adding an additional proximal or distal drive; how does this affect the simulation?

* View the evoked responses for different values of the scaling parameter; how does this affect the simulation?

* View the evoked responses for different values of the smoothing parameter; how does this affect the simulation?

* View the average evoked response generated by running more trials in a simulation; how does this affect the simulation?

* Try importing another network from the [templates provided](https://jonescompneurolab.github.io/hnn-core/stable/api.html#network-models-hnn-core). Can you figure out how these models are different from the original model? Try running a simulation with these models.